## Pre-processing

In [1]:

from llama_index.readers.file import PyMuPDFReader
from pathlib import Path
import config as cfg
from llama_index.core import Settings, VectorStoreIndex, StorageContext
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import Document
import faiss
import os
from llama_index.llms.groq import Groq
import torch
from llama_index.core.node_parser import (
    HierarchicalNodeParser,
    SentenceSplitter,
)
from llama_index.core.node_parser import get_leaf_nodes, get_root_nodes

c:\Users\PascaM\projectMM\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_documents_from_directory(directory: Path) -> list:
    """Carica tutti i documenti PDF dalla directory specificata."""
    documents = []
    reader = PyMuPDFReader()
    
    # Itera su tutti i file PDF nella directory
    for file_path in directory.glob("*.pdf"):
        try:
            docs = reader.load_data(file_path)
            doc_text = "\n\n".join([d.get_content() for d in docs])
            doc_text = Document(text=doc_text)
            documents.append(doc_text)
            print(f"Caricato: {file_path.name}")
        except Exception as e:
            print(f"Errore nel caricamento di {file_path.name}: {e}")
    
    return documents

In [3]:


docs = load_documents_from_directory(cfg.DOCUMENTS_DIR)

Caricato: TICY-Linea guida tecniche di offuscamento dei dati e uso della crittografia.pdf
Caricato: TICY-Linee guida sullo sviluppo di software sicuro.pdf
Caricato: Tinexta Cyber-Vulnerability Management Policy.pdf
Caricato: Tinexta Cyber-Vulnerability Response Playbook.pdf
Caricato: TINEXTA CYBER_Piano di Risposta agli Incidenti - IRP.pdf


In [4]:


llm = Groq(model=cfg.LLM_MODEL,api_key=cfg.GROQ_API_KEY)

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
embed_model = HuggingFaceEmbedding(
    model_name="all-MiniLM-L6-v2",
    device=device
)


In [6]:
Settings.embed_model = embed_model


## Auto-merge retrieval (base)

In [7]:
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[512, 1024, 2048],
    chunk_overlap=100, 
       
)
nodes = node_parser.get_nodes_from_documents(docs)


leaf_nodes = get_leaf_nodes(nodes)
root_nodes = get_root_nodes(nodes)

In [8]:
docstore = SimpleDocumentStore()

# insert nodes into docstore
docstore.add_documents(nodes)

In [9]:
dimension = len(embed_model.get_query_embedding("test"))
faiss_index = faiss.IndexFlatL2(dimension)
vector_store = FaissVectorStore(faiss_index=faiss_index)
    
    # Creiamo un contesto di archiviazione con il vector store
storage_context = StorageContext.from_defaults(
    vector_store=vector_store,
    docstore=docstore,
    )
    
    # Creazione dell'indice vettoriale
vector_index = VectorStoreIndex(
        leaf_nodes,
        storage_context=storage_context,
    )
    
    # Otteniamo il retriever base
base_retriever = vector_index.as_retriever(similarity_top_k=3)
    
    # Creiamo l'Auto Merging Retriever
auto_merging_retriever = AutoMergingRetriever(
        simple_ratio_thresh=0.5,
        vector_retriever=base_retriever,
        storage_context=storage_context,
        verbose=True
    )

In [10]:
from llama_index.core.response.notebook_utils import display_source_node
response = auto_merging_retriever.retrieve("È presente una policy aziendale relativa alla gestione degli audit all'interno dell'indice dei documenti?")
for node in response:
    display_source_node(node, source_length=10000)

> Merging 1 nodes into parent node.
> Parent node id: 16cace61-a02a-40cd-82c8-1f71ec03ab3b.
> Parent node text: • Gli audit log devono registrare il verificarsi di un evento, 
catturando informazioni come l'or...

> Merging 1 nodes into parent node.
> Parent node id: 41c3da5f-842e-4865-837f-253eff89f16f.
> Parent node text: Per verificare la qualità e 
l'accuratezza della documentazione prodotta e per identificare e ris...

> Merging 1 nodes into parent node.
> Parent node id: 223cb7d6-c916-4a1d-b7ba-987e5183b317.
> Parent node text: 1 Tale funzionalità è utile per le issue correlate tra loro, poiché consente in modo flessibile l...

> Merging 1 nodes into parent node.
> Parent node id: 969a448d-f76a-475c-9a45-86de7a6bb899.
> Parent node text: • Gli audit log devono registrare il verificarsi di un evento, 
catturando informazioni come l'or...

> Merging 1 nodes into parent node.
> Parent node id: 7365fb0f-4064-4c29-8f05-7d15b1355190.
> Parent node text: Per verificare la qualità e 
l'a

**Node ID:** 567616ba-9617-470a-ac62-bb2d2686f81e<br>**Similarity:** 0.8310713768005371<br>**Text:** 1 Tale funzionalità è utile per le issue correlate tra loro, poiché consente in modo flessibile la definizione di dipendenze 
a monte o a valle tra issue, es. in relazione parent/child e viceversa, via inserimento dell’issue id nel campo apposito.

Vulnerability Response Playbook 
Codice Procedura: 
PBVRESP 
Rev. 01 
Pag. 32 di 51 
Riservatezza: 
Interno Tinexta Cyber SpA 
 
 
 
Fig. 9 – Esempio di post su Canale Teams con subject 
Nota 
Gli scopi del Registro Eventi e del Canale Teams sono differenti: 
• 
Il Registro Eventi è il luogo dove tracciare, anche a fini di possibili audit, e catalogare come issue le 
azioni che richiedono una qualche forma di intervento o di remediation, fino alla loro chiusura. 
• 
Il Canale Teams è uno strumento per 
o condividere tra più Aree informazioni sulle valutazioni e correzioni intraprese; 
o coordinare le azioni in un ambiente “friendly”, evitando dispersivi giri di email, es. nel caso 
di vulnerabilità con impatti su più Aree, per procedere, in caso di mitigazione, con un rilascio 
campione da parte di una singola Area e successivamente da parte delle rimanenti. 
3.5.4 Misurazione delle Vulnerabilità e Valutazione delle Minacce  
Saper “leggere” le vulnerabilità, cioè disporre delle informazioni più corrette e complete, è fondamentale 
nella valutazione dei rischi a esse associate, oltre a essere necessario al fine di effettuare gli interventi di 
risoluzione così come descritto per il processo di risposta. Nel seguito è indicato qual è il sistema di 
misurazione da utilizzare nel processo di risposta e quali sono le sue modalità di impiego.<br>

**Node ID:** 7365fb0f-4064-4c29-8f05-7d15b1355190<br>**Similarity:** 0.80426025390625<br>**Text:** Per verificare la qualità e 
l'accuratezza della documentazione prodotta e per identificare e risolvere eventuali lacune, errori, 
ambiguità o conflitti, si può ricorrere a tecniche come revisioni, ispezioni, walkthrough e audit. Si 
possono anche usare strumenti come simulazione, prototipazione e testing, per confermare che i 
requisiti di sicurezza siano fattibili e realizzabili e per misurare e valutare le loro prestazioni e la loro 
efficacia. 
6. Gestione: avviene durante tutto il ciclo di vita e non è limitata all’analisi, per garantire che i requisiti 
di sicurezza siano aggiornati, comunicati e implementati correttamente. È quindi più un tema di 
pianificazione, organizzazione delle attività e utilizzo di strumenti, es. utilizzando strumenti di 
ticketing, di controllo versione e di Configuration and Change Management dovrebbero essere 
tracciate e controllate anche le modifiche e le revisioni dei requisiti di sicurezza, mantenendone 
l'integrità con il software versionato. Si possono inoltre usare piattaforme di collaborazione, 
strumenti di reporting o altri meccanismi di riscontro per agevolare la condivisione e lo scambio di 
informazioni tra il team di progetto e le altre parti interessate, nonché per monitorare e migliorare 
la soddisfazione dei requisiti di sicurezza e la loro conformità. 
Come descritto brevemente nel seguito, l’analisi deve includere: 
• requisiti di sicurezza funzionali; 
• requisiti di sicurezza non funzionali; 
• requisiti di sviluppo sicuro, suddivisi in 
o requisiti di sicurezza tecnici, che comprendono requisiti di raffinamento e requisiti architetturali; 
o requisiti di codifica sicura, dipendenti dal linguaggio; 
• comprensione della superficie di attacco e  
• valutazione del rischio.<br>

**Node ID:** 969a448d-f76a-475c-9a45-86de7a6bb899<br>**Similarity:** 0.7745314836502075<br>**Text:** • Gli audit log devono registrare il verificarsi di un evento, 
catturando informazioni come l'ora in cui si è verificato, 
l'utente o il servizio responsabile, es.: 
o identificativo, descrizione e timestamp dell'evento; 
o attore (es. ID utente o ID API) e azione eseguita 
(creazione, modifica, cancellazione, ecc.; 
o elemento impattato (es. indirizzo IP e identificativo 
dell’applicazione, dispositivo, sistema, ecc.); 
o fonte da cui proviene l'attore (paese, nome host, 
indirizzo IP, ID dispositivo, ecc.); 
o eventuali tag personalizzati, come il livello di gravità 
attribuito all'evento (es. clear, warning, minor, 
major, critical). 
Asset 
Management 
• Onde scongiurare fenomeni di “Shadow 
IT”, deve essere garantito e documentato 
l’inserimento, nel’Asset Inventory 
aziendale, dell’applicazione e 
dell’ambiente in cui essa opera, di 
produzione o meno. 
• L’inserimento nell’Asset Inventory aziendale deve 
includere il nominativo di chi ha la titolarità (ownership) 
dell’applicazione / ambiente, ovvero della funzione o 
ruolo che ha la responsabilità primaria dell’attività, della 
produttività e della resilienza dell’asset in questione. 
• Questo principio è essenziale per la risposta alle 
vulnerabilità e la gestione continua delle stesse, dato che 
il processo aziendale di risposta a vulnerabilità presenti su 
un determinato “Asset” risulterebbe bloccato nel caso 
non fosse preventivamente individuato il relativo “Asset 
Owner”, cfr. [41].

Linee guida  
sullo sviluppo di software sicuro 
Codice Procedura: 
LGSWSIC 
Rev. 03 
Pag.<br>

## Query expansion

In [ ]:
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.core.indices.query.query_transform.base import StepDecomposeQueryTransform
from llama_index.core.query_engine import MultiStepQueryEngine
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.prompts import PromptTemplate

In [ ]:
Settings.llm = llm

In [ ]:
DIRECT_QUERY_TEMPLATE = """
Sei un assistente di ricerca che aiuta a decomporre domande complesse in sottodomande più semplici in italiano. Ogni sottodomanda deve includere informazioni contenute nella domanda originale.

Contesto sulla fonte di conoscenza:
{index_summary}

Domanda originale: {query}

La tua risposta deve contenere SOLO una sottodomanda specifica in italiano, senza spiegazioni o ragionamenti. 
Non includere frasi come "Ecco una sottodomanda" o "La sottodomanda è".
Rispondi con una singola frase interrogativa.
"""

In [ ]:
step_decompose_transform = StepDecomposeQueryTransform(
    llm=llm,
    verbose=True,
    #step_decompose_query_prompt=PromptTemplate(DIRECT_QUERY_TEMPLATE),
    )

In [ ]:


# Crea il query engine direttamente con il tuo retriever
base_query_engine = RetrieverQueryEngine.from_args(
    retriever=auto_merging_retriever,
    llm=llm
)
index_summary = "Questo indice contiene documenti PDF relativi a policy aziendali di vario tipo. Gli articoli sono stati caricati da un dataset di documenti PDF e indicizzati con un indice vettoriale. Il motore di ricerca è in grado di rispondere a domande relative ai contenuti dei documenti."

In [ ]:

query_engine = MultiStepQueryEngine(
    query_engine=base_query_engine,
    query_transform=step_decompose_transform,
    index_summary=index_summary
)

response = query_engine.query('''È stato stabilito, documentato, approvato, comunicato, applicato, valutato e mantenuto un piano d'azione correttivo basato sul rischio per rimediare ai risultati dell'audit?''')
print(response)


## Keyword extraction for metadata

### KeyBert

In [ ]:
from keybert import KeyBERT
import torch
from typing import List
from sentence_transformers import SentenceTransformer

# Controlla se è disponibile la GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Dispositivo in uso: {device}")

# Inizializza il modello Sentence Transformer con GPU (KeyBERT usa questo sotto il cofano)
model_name = "all-mpnet-base-v2"  # Questo modello è multilingue e leggero
sentence_model = SentenceTransformer(model_name, device=device)

if device == "cuda":
    print(f"Utilizzando la GPU: {torch.cuda.get_device_name(0)}")

# Inizializza KeyBERT con il modello precaricato su GPU
kw_model = KeyBERT(model=sentence_model)

# Esempio di chunk di testo
chunk = """
Gli audit log devono registrare il verificarsi di un evento, catturando informazioni come l'ora in cui si è verificato, l'utente o il servizio responsabile, es.: o identificativo, descrizione e timestamp dell'evento; o attore (es. ID utente o ID API) e azione eseguita (creazione, modifica, cancellazione, ecc.; o elemento impattato (es. indirizzo IP e identificativo dell’applicazione, dispositivo, sistema, ecc.); o fonte da cui proviene l'attore (paese, nome host, indirizzo IP, ID dispositivo, ecc.); o eventuali tag personalizzati, come il livello di gravità attribuito all'evento (es. clear, warning, minor, major, critical). Asset Management • Onde scongiurare fenomeni di “Shadow IT”, deve essere garantito e documentato l’inserimento, nel’Asset Inventory aziendale, dell’applicazione e dell’ambiente in cui essa opera, di produzione o meno. • L’inserimento nell’Asset Inventory aziendale deve includere il nominativo di chi ha la titolarità (ownership) dell’applicazione / ambiente, ovvero della funzione o ruolo che ha la responsabilità primaria dell’attività, della produttività e della resilienza dell’asset in questione. • Questo principio è essenziale per la risposta alle vulnerabilità e la gestione continua delle stesse, dato che il processo aziendale di risposta a vulnerabilità presenti su un determinato “Asset” risulterebbe bloccato nel caso non fosse preventivamente individuato il relativo “Asset Owner”, cfr. [41].

Linee guida
sullo sviluppo di software sicuro Codice Procedura: LGSWSIC Rev. 03 Pag.
"""

# Estrai parole chiave direttamente
keywords = kw_model.extract_keywords(
    chunk,
    keyphrase_ngram_range=(1, 2),  # Considerare unigrammi e bigrammi          # KeyBERT gestisce automaticamente anche per testi in italiano
    use_mmr=True,                  # Usa Maximal Marginal Relevance per diversificare
    diversity=0.7,                 # Bilanciamento tra rilevanza e diversità
    top_n=3                       # Numero di parole chiave da estrarre
)

# Stampa le parole chiave estratte con i loro punteggi
print("\nParole chiave estratte con punteggi:")
for keyword, score in keywords:
    print(f"- {keyword}: {score:.4f}")

# Estrai solo le parole chiave (senza punteggi)
keyword_list = [keyword for keyword, _ in keywords]

# Crea un Document di LlamaIndex con le parole chiave nei metadati
document = Document(
    text=chunk,
    doc_id="doc1",
    metadata={"keywords": keyword_list}
)

print(f"\nDocument creato con ID: {document.doc_id}")
print(f"Metadati con parole chiave: {document.metadata}")

### SpaCy

In [ ]:
import spacy
from collections import Counter
from string import punctuation
nlp = spacy.load("it_core_news_sm")
def get_hotwords(text):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN'] 
    doc = nlp(text.lower()) 
    for token in doc:
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        if(token.pos_ in pos_tag):
            result.append(token.text)
    return result
text = """
Gli audit log devono registrare il verificarsi di un evento, catturando informazioni come l'ora in cui si è verificato, l'utente o il servizio responsabile, es.: o identificativo, descrizione e timestamp dell'evento; o attore (es. ID utente o ID API) e azione eseguita (creazione, modifica, cancellazione, ecc.; o elemento impattato (es. indirizzo IP e identificativo dell’applicazione, dispositivo, sistema, ecc.); o fonte da cui proviene l'attore (paese, nome host, indirizzo IP, ID dispositivo, ecc.); o eventuali tag personalizzati, come il livello di gravità attribuito all'evento (es. clear, warning, minor, major, critical). Asset Management • Onde scongiurare fenomeni di “Shadow IT”, deve essere garantito e documentato l’inserimento, nel’Asset Inventory aziendale, dell’applicazione e dell’ambiente in cui essa opera, di produzione o meno. • L’inserimento nell’Asset Inventory aziendale deve includere il nominativo di chi ha la titolarità (ownership) dell’applicazione / ambiente, ovvero della funzione o ruolo che ha la responsabilità primaria dell’attività, della produttività e della resilienza dell’asset in questione. • Questo principio è essenziale per la risposta alle vulnerabilità e la gestione continua delle stesse, dato che il processo aziendale di risposta a vulnerabilità presenti su un determinato “Asset” risulterebbe bloccato nel caso non fosse preventivamente individuato il relativo “Asset Owner”, cfr. [41].

Linee guida
sullo sviluppo di software sicuro Codice Procedura: LGSWSIC Rev. 03 Pag.
"""
output = set(get_hotwords(text))
most_common_list = Counter(output).most_common(10)
for item in most_common_list:
  print(item[0])

### Yake

In [ ]:

from yake import KeywordExtractor

# Create a KeywordExtractor instance
kw_extractor = KeywordExtractor()

# Text from which keywords will be extracted
text = '''Gli audit log devono registrare il verificarsi di un evento, catturando informazioni come l'ora in cui si è verificato, l'utente o il servizio responsabile, es.: o identificativo, descrizione e timestamp dell'evento; o attore (es. ID utente o ID API) e azione eseguita (creazione, modifica, cancellazione, ecc.; o elemento impattato (es. indirizzo IP e identificativo dell’applicazione, dispositivo, sistema, ecc.); o fonte da cui proviene l'attore (paese, nome host, indirizzo IP, ID dispositivo, ecc.); o eventuali tag personalizzati, come il livello di gravità attribuito all'evento (es. clear, warning, minor, major, critical). Asset Management • Onde scongiurare fenomeni di “Shadow IT”, deve essere garantito e documentato l’inserimento, nel’Asset Inventory aziendale, dell’applicazione e dell’ambiente in cui essa opera, di produzione o meno. • L’inserimento nell’Asset Inventory aziendale deve includere il nominativo di chi ha la titolarità (ownership) dell’applicazione / ambiente, ovvero della funzione o ruolo che ha la responsabilità primaria dell’attività, della produttività e della resilienza dell’asset in questione. • Questo principio è essenziale per la risposta alle vulnerabilità e la gestione continua delle stesse, dato che il processo aziendale di risposta a vulnerabilità presenti su un determinato “Asset” risulterebbe bloccato nel caso non fosse preventivamente individuato il relativo “Asset Owner”, cfr. [41].

Linee guida
sullo sviluppo di software sicuro Codice Procedura: LGSWSIC Rev. 03 Pag.'''

# Extract keywords from the text
keywords = kw_extractor.extract_keywords(text)

# Print the extracted keywords and their scores
for kw in keywords:
    print("Keyword:", kw[0], "Score:", kw[1])

### Rake

In [ ]:
# installation


import nltk
nltk.download('stopwords')
nltk.download('punkt')
from rake_nltk import Rake

# Create a Rake instance
r = Rake()

# Text from which keywords will be extracted
text = '''Gli audit log devono registrare il verificarsi di un evento, catturando informazioni come l'ora in cui si è verificato, l'utente o il servizio responsabile, es.: o identificativo, descrizione e timestamp dell'evento; o attore (es. ID utente o ID API) e azione eseguita (creazione, modifica, cancellazione, ecc.; o elemento impattato (es. indirizzo IP e identificativo dell’applicazione, dispositivo, sistema, ecc.); o fonte da cui proviene l'attore (paese, nome host, indirizzo IP, ID dispositivo, ecc.); o eventuali tag personalizzati, come il livello di gravità attribuito all'evento (es. clear, warning, minor, major, critical). Asset Management • Onde scongiurare fenomeni di “Shadow IT”, deve essere garantito e documentato l’inserimento, nel’Asset Inventory aziendale, dell’applicazione e dell’ambiente in cui essa opera, di produzione o meno. • L’inserimento nell’Asset Inventory aziendale deve includere il nominativo di chi ha la titolarità (ownership) dell’applicazione / ambiente, ovvero della funzione o ruolo che ha la responsabilità primaria dell’attività, della produttività e della resilienza dell’asset in questione. • Questo principio è essenziale per la risposta alle vulnerabilità e la gestione continua delle stesse, dato che il processo aziendale di risposta a vulnerabilità presenti su un determinato “Asset” risulterebbe bloccato nel caso non fosse preventivamente individuato il relativo “Asset Owner”, cfr. [41].

Linee guida
sullo sviluppo di software sicuro Codice Procedura: LGSWSIC Rev. 03 Pag.'''

# Extract keywords from the text
r.extract_keywords_from_text(text)

# Get the ranked keywords
keywords = r.get_ranked_phrases_with_scores()

# Print the extracted keywords and their scores
for score, kw in keywords:
    print("Keyword:", kw, "Score:", score)

### LLM based

In [11]:
from langchain_ollama import ChatOllama
import json
def keyword_extraction(text:str):
    local_llm = "llama3.2:latest"
    llm_json_mode = ChatOllama(model=local_llm, temperature=0, format="json")
    prompt = f"""Estrai tre keywords da questo testo, cercando di mantenere il significato originale, 
    e dammele in output in formato JSON.
    Le keyword vanno date in output prendendo ad esempio lo schema di questo esempio:
    {{'keywords': ['keyword 1', 'keyword 2', 'keyword 3']}}.
    
    Ricorda questo che hai letto è un esempio e non un output fisso da dare.
    Qui di seguito trovi il testo da cui devi estrarle: 
    {text}
    """

    #prompt_formatted = prompt.format(text=text)
    response = llm_json_mode.invoke(prompt)
    return response



## Retrieval with metadata

In [21]:
import torch
from typing import List
from llama_index.core.node_parser import HierarchicalNodeParser, get_leaf_nodes,get_root_nodes



def create_nodes_with_metadata(docs):
    """
    Crea i nodi partendo da un 'Document' o una lista di Document, 
    aggiunge metadati 'keywords' usando la funzione keyword_extraction.
    Restituisce una tupla: (tutti_i_nodi, nodi_foglia).
    """

    # Parser per creare i nodi
    node_parser = HierarchicalNodeParser.from_defaults(
        chunk_sizes=[512, 1024,2048],
        chunk_overlap=100
    )
    all_nodes = node_parser.get_nodes_from_documents(docs)
    leaf_nodes = get_leaf_nodes(all_nodes)

    # Funzione helper per estrarre le keyword (JSON)
    def get_keywords_from_chunk(chunk_text: str):
        response = keyword_extraction(chunk_text)
        try:
            data = json.loads(response.content)
            # Supponiamo che il JSON abbia la struttura {"keywords": [...]} 
            if isinstance(data, dict) and "keywords" in data:
                return data["keywords"]
            return []
        except json.JSONDecodeError:
            return []

    # Aggiunta dei metadati "keywords" a ciascun nodo foglia
    for node in leaf_nodes:
        chunk_text = node.get_content()
        keywords = get_keywords_from_chunk(chunk_text)
        node.metadata["keywords"] = keywords
        print(node.metadata)

    return all_nodes, leaf_nodes

In [22]:
nodes,leaf_nodes = create_nodes_with_metadata(docs)

    

{'keywords': ['Offuscamento dei dati', 'Crittografia', 'Data Protection']}
{'keywords': ['pseudonimizzazione', 'cifratura', 'anonimizzazione']}
{'keywords': ['Riservatezza', 'Crittografia', 'Key Management']}
{'keywords': ['Key Management', 'Crittografia', 'Tecniche di offuscamento dei dati']}
{'keywords': ['pseudonimizzazione', 'dati protetti', 'garante privacy']}
{'keywords': ['Crittografia', 'Anonimizzazione', 'Protezione dei dati']}
{'keywords': ['GDPR', 'anonymisation', 'progettazione sicura']}
{'keywords': ['Privacy by Design', 'Differential Privacy', 'OpenDP']}
{'keywords': ['Anonimizzazione', 'Cifrario', 'Attacco al canale laterale']}
{'keywords': ['criptografia', 'codice segreto', 'confidenzialità']}
{'keywords': ['Crittografia', 'Data Controller', 'Data Processor']}
{'keywords': ['Data Protection', 'Privacy by Design', 'GDPR']}
{'keywords': ['DPO', 'Trattamento dati personali', "Diritto all'oblio"]}
{'keywords': ['Diritto di rettifica', 'De-identificazione', 'DES']}
{'keyword

In [12]:
# Salva i nodi in formato JSON
with open('nodes_data.json', 'w', encoding='utf-8') as f:
    # Converti i nodi in dizionari per la serializzazione
    nodes_dict = [node.to_dict() for node in leaf_nodes]
    json.dump(nodes_dict, f, ensure_ascii=False, indent=2)

In [13]:
import json
from llama_index.core.schema import Node

# Carica i nodi dal file JSON
with open('nodes_data.json', 'r', encoding='utf-8') as f:
    nodes_dict = json.load(f)
    # Converti i dizionari in oggetti Node
    leaf_2 = [Node.from_dict(node_data) for node_data in nodes_dict]

In [ ]:
for idx,node in enumerate(leaf_2):
    leaf_nodes[idx].metadata = node.metadata
    print(leaf_nodes[idx].metadata)


## Sparse retreiver

In [11]:
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer

# We can pass in the index, docstore, or list of nodes to create the retriever
bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=10,
    # Optional: We can pass in the stemmer and set the language for stopwords
    # This is important for removing stopwords and stemming the query + text
    # The default is english for both
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

resource module not available on Windows


In [12]:
from llama_index.core.response.notebook_utils import display_source_node

# will retrieve context from specific companies
retrieved_nodes = bm25_retriever.retrieve(
    "Le politiche e le procedure di gestione della continuità operativa e di resilienza operativa sono stabilite, documentate, approvate, comunicate, applicate a tutti i progetti, valutate e mantenute?"
)
for node in retrieved_nodes:
    display_source_node(node, source_length=10000)

**Node ID:** e81e5113-dfed-4a3f-95cd-4d6ce8d747fe<br>**Similarity:** 9.290040969848633<br>**Text:** [1];  
• 
descrive i collegamenti operativi e comunicativi tra i team interni di Tinexta Cyber, il GSOC (Group 
SOC di Tinexta SpA) e le altre entità di Gruppo. 
1.3 CAMPO DI APPLICAZIONE 
Il presente documento si applica all’intera Azienda. 
1.4 RIFERIMENTI 
Tabella 1 - Elenco dei Riferimenti e delle Citazioni 
Rif. 
URL  / ISBN / Titolo 
Descrizione 
[1] 
 
NIST SP 800-61 Revision 2 
NIST Computer Security Incident Handling Guide. 
[2] 
 
Tinexta Cyber Spa-Sicurezza e 
Protezione dati 
Manuale di sicurezza e protezione dei dati personali 
• Parte I: 
Policy generale e organizzazione; 
• Parte II: Misure e procedure di sicurezza adottate; 
• Parte III: Linee guida per la progettazione e l’esecuzione dei 
trattamenti di dati personali; 
• Parte IV: Norme per il corretto utilizzo degli strumenti informatici e 
dei dati aziendali. 
 
[3] 
 
Tinexta Cyber Spa-Piano 
Continuità Operativa Servizi IT 
Piano di Continuità Operativa - Servizi di base: controllo del dominio, 
gestione della posta elettronica, accesso ad Internet, VPN site-to-site. 
[4] 
 
Istr.Op.39 Gestione delle 
violazioni di dati personali 
Piano di risposta alle violazioni – data breach - che si possono verificare 
durante il trattamento dei dati personali.

Piano di Risposta agli Incidenti 
(Incident Response Plan) 
Codice Procedura: 
IRP 
Rev. 01 
Pag. 5 di 36 
Riservatezza: 
Interno Tinexta Cyber SpA 
ISO/IEC 27001 
 
1.5 GLOSSARIO DEI TERMINI 
Per alcune definizioni è stata mantenuta la dicitura originale in inglese.<br>

**Node ID:** 96a969e5-3c87-4671-8721-c0ca713739ea<br>**Similarity:** 8.362198829650879<br>**Text:** ); 
● gli eventi compromettenti i software applicativi dei clienti e fornitori di cui Tinexta Cyber si serve per 
l’erogazione dei servizi ai clienti; 
● gli eventi compromettenti i dati aziendali e/o i dati personali, siano essi dei dipendenti/collaboratori 
che dei clienti finali delle società a cui Tinexta Cyber eroga servizi. 
Sono invece esclusi dal campo di applicazione della presente procedura gli eventi disastrosi che rientrano 
nell’ambito della continuità operativa. Tali situazioni sono gestite secondo le modalità descritte nel Piano di 
Risposta agli Incidenti (IRP), cfr. cap. 1 ÷ 6 di questo documento, nonché nel Piano di Continuità Operativa 
Servizi IT, cfr. [3]. 
7.3.2 Modalità Operative 
Per la gestione degli incidenti di sicurezza è adottato il software applicativo Mantis Bug Tracker, di seguito 
anche MantisBT, denominato specificamente per l’area InVita Help-ITO.  
Tramite tale strumento sono registrati sia gli incidenti di sicurezza sia le richieste di servizio. Il software è 
stato infatti adattato alle esigenze di Tinexta Cyber mediante l’adozione di una configurazione ad hoc.  
Oltre al menzionato MantisBT è previsto, per le emergenze, un numero di telefono fisso e di cellulare riferito 
agli Amministratori di Sistema Tinexta Cyber. Gli eventuali incidenti di sicurezza/richieste di servizio in 
emergenza, per cui è impossibile per il segnalatore aprire il ticket, sono comunque censiti in MantisBT 
direttamente dagli Amministratori di Sistema.  
Le categorie di segnalazioni previste dallo strumento sono elencate nel seguito.

Piano di Risposta agli Incidenti 
(Incident Response Plan) 
Codice Procedura: 
IRP 
Rev. 01 
Pag.<br>

**Node ID:** c16c1d91-aa9a-4cd3-8b1b-4a237d866383<br>**Similarity:** 7.028223037719727<br>**Text:** Attacco 
Con tale termine ci riferiamo in questo documento a un tentativo malevolo e intenzionale 
da parte di un individuo o di un'organizzazione di violare il sistema informativo dell’Azienda 
per ottenere un qualche tipo di vantaggio o visibilità.

Vulnerability Response Playbook 
Codice Procedura: 
PBVRESP 
Rev. 01 
Pag. 8 di 51 
Riservatezza: 
Interno Tinexta Cyber SpA 
 
 
Termine 
Descrizione 
Audit 
Con tale termine ci riferiamo in questo documento a revisioni tecniche, di carattere più 
ampio e sistematico rispetto agli assessment, per controllare l’efficacia e la correttezza 
delle misure definite, realizzate e mantenute a garanzia della sicurezza informatica e delle 
informazioni in Azienda 
BCP 
Business Continuity Plan, o Piano di Continuità Operativa (PCO), ovvero la documentazione 
che delinea come un'azienda continuerà ad operare durante un'interruzione non 
pianificata del servizio. È più completo di un piano di disaster recovery e comprende piani 
di emergenza per i processi aziendali, i beni, le risorse umane e i partner commerciali - ogni 
aspetto del business che potrebbe essere colpito. 
Bug 
Un errore di implementazione. 
CERT 
Computer Emergency Response Team, organizzazioni generalmente gestite e finanziate da 
Università o Enti Governativi nazionali o sovranazionali (es. CERT IT, CERT EU), incaricate di 
individuare, raccogliere e trattare le segnalazioni di incidenti informatici e potenziali 
vulnerabilità nei software. 
CNA 
CVE Numbering Authority, software vendor, team open source, gruppi di ricerca e altri 
soggetti autorizzati dal Programma CVE ad assegnare ID CVE alle vulnerabilità e pubblicare 
record CVE all'interno del proprio ambito applicativo. 
CSIRT 
Computer Security Incident Response Team, strutture generalmente gestite e finanziate da 
Enti Governativi nazionali (es.<br>

**Node ID:** 6c93aa7c-d0fe-406a-94a5-817bc08df16a<br>**Similarity:** 6.878396987915039<br>**Text:** 01 
Pag. 16 di 36 
Riservatezza: 
Interno Tinexta Cyber SpA 
ISO/IEC 27001 
 
➢ GSOC: il SOC di Gruppo (GSOC)   che fornisce il monitoraggio di sicurezza di primo e secondo livello, vedi 
figura seguente. 
 
 
Fig. 2 – Schematizzazione del SOC di Gruppo 
La matrice RACI riportata nell’Allegato A del presente documento riporta le responsabilità dell’IMT, IRT e del 
GSOC per ciascuna fase. 
 
 
3.1.2 Identificazione dei tool necessari 
L’azienda deve avere più meccanismi di comunicazione e coordinamento che siano separati e indipendenti 
da quelli utilizzati quotidianamente, al fine di garantire la continuità in caso di interruzione operativa dovuta 
all’incidente in atto. Per questa ragione l’IRT dispone degli strumenti dedicati descritti nella tabella seguente. 
 
Tabella 5 – Strumenti dedicati. 
STRUMENTO 
DESCRIZIONE 
Microsoft Teams aziendale 
Canali di comunicazione veloce. 
Atlassian JIRA Cloud 
Software per il tracciamento delle segnalazioni. 
Signal 
Software per comunicazioni criptate out-of-band. 
Connettività di backup 
Rete non collegata all’infrastruttura sotto attacco, qualora fosse necessario isolare 
l’infrastruttura da internet, ma allo stesso tempo provvedere a una capacità operativa 
per i team coinvolti nella gestione dell’incidente. 
 
 
cloud 
• L1 & L2 Teams 
work autonomously 
• another Team  
is kept in standby  
just in case

Piano di Risposta agli Incidenti 
(Incident Response Plan) 
Codice Procedura: 
IRP 
Rev. 01 
Pag. 17 di 36 
Riservatezza: 
Interno Tinexta Cyber SpA 
ISO/IEC 27001 
 
3.2 DETECTION & ANALYSIS 
 
Fig.<br>

In [38]:
from llama_index.core.node_parser import SimpleNodeParser
import spacy

# Carica il modello italiano di spaCy
nlp = spacy.load('it_core_news_sm')

# Funzione di preprocessing per l'italiano
def preprocess_text(text):
    doc = nlp(text)
    # Rimuovi stopwords e lemmatizza
    tokens = [token.lemma_.lower() for token in doc 
             if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Configura BM25Retriever con preprocessamento italiano
bm25_retriever = BM25Retriever.from_defaults(
    nodes=leaf_nodes,
    tokenizer=lambda x: preprocess_text(x).split(),  # Custom tokenizer
    similarity_top_k=10
)

The tokenizer parameter is deprecated and will be removed in a future release. Use a stemmer from PyStemmer instead.


## Ensamble retreiver

In [63]:
llm = Groq(model=cfg.LLM_KEYWORD,api_key=cfg.GROQ_API_KEY)

In [75]:
from llama_index.core.retrievers import QueryFusionRetriever

retriever = QueryFusionRetriever(
    [auto_merging_retriever, bm25_retriever],
    similarity_top_k=3,
    num_queries=4,  # set this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    llm=llm
    # query_gen_prompt="...",  # we could override the query generation prompt here
)

In [76]:
# apply nested async to run in a notebook
import nest_asyncio

nest_asyncio.apply()
nodes_with_scores = retriever.retrieve(
    "Le politiche e le procedure di gestione della continuità operativa e di resilienza operativa sono stabilite, documentate, approvate, comunicate, applicate a tutti i progetti, valutate e mantenute?"
)

Generated queries:
Here are three search queries based on your input:
1. Business continuity management policies and procedures documentation
2. Operational resilience framework implementation in projects
3.  Evaluating and maintaining business continuity and resilience plans
Let me know if you'd like me to generate more!
> Merging 1 nodes into parent node.
> Parent node id: e37ed5ad-b2aa-4db6-8e40-c667ae075639.
> Parent node text: • Deve essere prodotta una lista contenente, per ogni issue 
riscontrata, informazioni utili alla...

> Merging 1 nodes into parent node.
> Parent node id: 2cf04044-e6bc-476b-9225-3833b26e47bb.
> Parent node text: L’obiettivo di questa policy è pertanto stabilire le regole per la revisione, valutazione, applic...

> Merging 1 nodes into parent node.
> Parent node id: 2074ba4f-750d-4be7-9b7f-fa69480acd52.
> Parent node text: E, ancora più importante, quanto possono essere felici i Clienti di avere nuove funzionalità aggi...

> Merging 1 nodes into parent node

In [77]:
for node in nodes_with_scores:
    print(f"Score: {node.score:.2f} - {node.text}...\n-----\n")

Score: 0.03 - 5.9.2 Continuous development automation 
Tabella 30 – Continuous Integration, Continuous Delivery e Continuous Deployment, alcuni esempi. 
Item 
URL 
Descrizione 
UML, UMLSec 
http://www.omg.org/spec
/UML  
Unified Modeling Language, il cui formalismo è un vero e proprio standard 
(ISO/IEC 19501, se si usano le specifiche OMG-UML v1.4.2; oppure ISO/IEC 19505, 
se si usano le specifiche OMG-UML v2.4.1). 
UMLSec è un'estensione di UML, che integra le informazioni sulla sicurezza 
mediante una serie di stereotypes specifici («fair exchange», «guarded access», 
«no misuse», «rbac», ecc.). 
BPMN 
https://www.omg.org/spe
c/BPMN  
Business Process Model & Notation, il cui formalismo è un vero e proprio standard 
(ISO/IEC 19510), al pari del UML. 
DFD 
- 
Data Flow Diagram, uno standard de facto per rappresentare il flusso dei dati, con 
almeno tre formalismi differenti: Yourdon e Coad, Yourdon e DeMarco, Gane e 
Sarson. I DFD sono ampiamente utilizzati in  diverse metodologie di

In [78]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever=retriever, llm=llm)
response = query_engine.query("Le politiche e le procedure di gestione della continuità operativa e di resilienza operativa sono stabilite, documentate, approvate, comunicate, applicate a tutti i progetti, valutate e mantenute?")

Generated queries:
Here are three search queries based on your input:
1. Business continuity management policies and procedures documentation
2. Operational resilience framework implementation in projects
3.  Evaluating and maintaining business continuity and resilience plans
Let me know if you'd like me to generate more!
> Merging 1 nodes into parent node.
> Parent node id: e37ed5ad-b2aa-4db6-8e40-c667ae075639.
> Parent node text: • Deve essere prodotta una lista contenente, per ogni issue 
riscontrata, informazioni utili alla...

> Merging 1 nodes into parent node.
> Parent node id: 2cf04044-e6bc-476b-9225-3833b26e47bb.
> Parent node text: L’obiettivo di questa policy è pertanto stabilire le regole per la revisione, valutazione, applic...

> Merging 1 nodes into parent node.
> Parent node id: 2074ba4f-750d-4be7-9b7f-fa69480acd52.
> Parent node text: E, ancora più importante, quanto possono essere felici i Clienti di avere nuove funzionalità aggi...

> Merging 1 nodes into parent node

In [79]:
from llama_index.core.response.notebook_utils import display_response

display_response(response)

**`Final Response:`** The provided text does not contain the answer to your question.